# Refine txt2img Prompts with Human Feedback


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CarperAI/trlx/blob/main/examples/notebooks/trlx_simulacra.ipynb)


#### Optimize a gpt2-based txt2img prompt generator to produce aesthetic prompts using https://github.com/JD-P/simulacra-aesthetic-captions

Notebook by [@smellslikeml](https://github.com/smellslikeml)

---

Execute the cells below to install [TRLX](https://github.com/CarperAI/trlx) for a colab environment.

In [1]:
!ls

RL_misko_trl.ipynb  trlx  trlx_simulacra.ipynb


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# !pip install torchtyping
# !python --version
# import deepspeed

In [4]:
# !git clone https://github.com/CarperAI/trlx.git
# !git config --global --add safe.directory /content/trlx && cd /content/trlx && pip install -e .

In [5]:
# uninstall scikit_learn + jax to avoid numpy issues
# !conda uninstall -y scikit_learn jax

In [2]:
import os

# run within repo
os.chdir('./trlx')
print(os.getcwd())

/mnt/storage-brno2/home/ahajek/Spektro/MassGenie/RLHF/trlx


In [5]:
import sqlite3
from urllib.request import urlretrieve

import trlx

url = "https://raw.githubusercontent.com/JD-P/simulacra-aesthetic-captions/main/sac_public_2022_06_29.sqlite"
dbpath = "sac_public_2022_06_29.sqlite"

if not os.path.exists(dbpath):
    print(f"fetching {dbpath}")
    urlretrieve(url, dbpath)

conn = sqlite3.connect(dbpath)
c = conn.cursor()
c.execute(
    "SELECT prompt, rating FROM ratings "
    "JOIN images ON images.id=ratings.iid "
    "JOIN generations ON images.gid=generations.id "
    "WHERE rating IS NOT NULL;"
)

prompts, ratings = tuple(map(list, zip(*c.fetchall())))

Trlx uses [wandb](https://wandb.ai/) to log results. Make sure to set up an account and use your token to authenticate when prompted after executing the cell below.

In [6]:
import trlx
trlx.train(
    "gpt2",
    samples=prompts,
    rewards=ratings,
    eval_prompts=["Hatsune Miku, Red Dress"] * 64,
)

/mnt/storage-brno2/home/ahajek/Spektro/MassGenie/RLHF/trlx/trlx/trlx.py:57: UserWarning: Passing the `config` argument implicitly is depreciated, use oradapt some from `trlx/data/default_configs.py` instead
  warnings.warn(
[RANK 0] Initializing model: gpt2


[RANK 0] Collecting rollouts
[RANK 0] Logging sample example


                                                  Sample Example                                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Prompt                     ┃ Response                                                                  ┃ Reward ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ <|endoftext|><|endoftext|> │ An artwork of a broken wine bottle in the medium of dry                   │ 7      │
│                            │ pigments<|endoftext|><|endoftext|>                                        │        │
└────────────────────────────┴───────────────────────────────────────────────────────────────────────────┴────────┘

[RANK 0] Logging experience string statistics


     Experience String Stats (mean ∈ [min, max])     
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Prompt Length ┃ Output Length   ┃ Sample Length   ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 2.00 ∈ [2, 2] │ 26.64 ∈ [2, 64] │ 28.64 ∈ [4, 66] │
└───────────────┴─────────────────┴─────────────────┘

...

[losses/loss: 4.04 | losses/loss_q: 0.32 | losses/loss_v: 0.05 | losses/loss_cql: 13.64 | losses/loss_awac: 2.30]: 100%|█████████▉| 999/1000 [02:09<00:00,  8.48it/s][RANK 0] Evaluating model
[generation sweep 1/1 | eval batch 2/2]: 100%|██████████| 2/2 [00:01<00:00,  1.69it/s]
[RANK 0] Summarizing evaluation


                                                  Evaluation #10                                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                  ┃ output                                                                                ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Hatsune Miku, Red Dress │ , Black Red, Gold, Art of the Neon, 4K, Hi-Fructose, CGSociety, CGsociety, CG, Art of │
│                         │ the Neon, CGSociety, Hi-Fructose, CGSociety, Hi-Fructose, Art                         │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ Hatsune Miku, Red Dress │  of an Angel, inks, Illustration, Illustrated, detailed, 4K, 4K UHD, Illustration,    │
│                         │ Illustration of a beautiful person,, a beautiful person, a cute, cute, cute,          │
│                         │ adorable, Cute, Cute, Cute,                                                           │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ Hatsune Miku, Red Dress │ , HD art, Hi-Fructose art, HD photography, Hi-Fructose Art, The art of high-quality   │
│                         │ art, HD CG, detailed and detailed, full-color, Hi-Fructose art, Art of the rainbow,   │
│                         │ realistic and realistic, Hi-F                                                         │
└─────────────────────────┴───────────────────────────────────────────────────────────────────────────────────────┘

[losses/loss: 4.68 | losses/loss_q: 0.34 | losses/loss_v: 0.05 | losses/loss_cql: 15.29 | losses/loss_awac: 2.76]: 100%|██████████| 1000/1000 [02:20<00:00,  3.36s/it][RANK 0] Evaluating model
[generation sweep 1/1 | eval batch 2/2]: 100%|██████████| 2/2 [00:01<00:00,  1.73it/s]
[RANK 0] Summarizing evaluation


                                                  Evaluation #11                                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                  ┃ output                                                                                ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Hatsune Miku, Red Dress │  of the Dragon, the Dragon, the Blizzard of light, and the blue sky, of the world, in │
│                         │ the style of the fantasy art.                                                         │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ Hatsune Miku, Red Dress │ , blue bliks, blue-ish, with pink, pinky, and green, with a look of hope on the face. │
│                         │ The colors of this look of hope is in the color pink, and the face is as bright as    │
│                         │ the clouds, and the clouds are a                                                      │
├─────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│ Hatsune Miku, Red Dress │ , Cinci, Cinquei, Cinquei of the universe, Cinquei of the future, 4k, Hi-Fructose,    │
│                         │ Cinquei of the past, HD Art, full-stain, Cinquei of the                               │
└─────────────────────────┴───────────────────────────────────────────────────────────────────────────────────────┘

[losses/loss: 4.68 | losses/loss_q: 0.34 | losses/loss_v: 0.05 | losses/loss_cql: 15.29 | losses/loss_awac: 2.76]: 100%|██████████| 1000/1000 [02:32<00:00,  6.57it/s]


In [7]:
["fwvver"] * 3

['fwvver', 'fwvver', 'fwvver']